# Cài đặt và import các module cần thiết

In [1]:
%pip install langchain-groq langchain langchain-community pypdf faiss-cpu sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.2/974.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 14.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cach

In [2]:
import os
import textwrap

import langchain
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, UnstructuredPDFLoader, YoutubeLoader, PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms import GPT4All
from langchain import LLMChain, PromptTemplate
from langchain_groq import ChatGroq



# Thiết lập call API của Grog và test mô hình trước khi thực hiện RAG

In [3]:
from groq import Groq

# Set the environment variable
os.environ['GROQ_API_KEY'] = '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'

# Initialize the client using the environment variable
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": """
<|im_start|>system
You are a Japanese history chatbot. You will receive a multi-choices question about Japanese history and you must present the right answer as well
as some explainations if possible

<|im_start|>user
### Question:
Who is the de-facto leader of the Western Army in 1600 :
### Choices:
A. Akechi Mitsuhide
B. Mori Terumoto
C. Tokugawa Ieyasu
D. Ishida Mitsunari
### Answer:

<|im_start|>assistant
""".strip(),
        }
    ],
    model="gemma-7b-it",
)

print(chat_completion.choices[0].message.content)


**Answer:** C. Tokugawa Ieyasu

**Explanation:**
Tokugawa Ieyasu played a pivotal role in the Battle of Sekigahara in 1600, defeating his rivals and establishing himself as the de-facto leader of the Western Army. His victory paved the way for the establishment of the Tokugawa Shogunate, which ruled Japan from 1603 to 1868.


# RAG

## Load data và thực hiện chia nhỏ thành các chunks

In [4]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("data.pdf")
pages = loader.load()

len(pages)

21

In [5]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

docs = text_splitter.split_documents(pages)
len(docs)

59

## Thực hiện embedding và lưu vào vector db

In [6]:
# Sử dụng mô hình embedding
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
hf_embeddings = HuggingFaceEmbeddings(model_name=MODEL_NAME)

# Chuyển toàn bộ text thông qua mô hình embedding về dạng vector và lưu dưới dạng db
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, hf_embeddings)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
db

## Thiết lập custom prompt và tạo chain

In [8]:
custom_prompt_template = """
You are a chatbot with extensive knowledge of Japanese history. Use the following information to answer the user's question.
If you don't know the answer, just say you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

"""

In [9]:
prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])

In [10]:
from langchain.prompts.prompt import PromptTemplate
from langchain_groq import ChatGroq
from langchain.memory import ChatMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

retriever = db.as_retriever()

model =ChatGroq(
            groq_api_key=os.environ.get("GROQ_API_KEY"),
            model_name="gemma-7b-it" )

chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    chain_type_kwargs={'prompt': prompt}
)


## Thử nghiệm mô hình sau khi RAG

In [11]:
response = chain.run("""Who is the de-facto leader of the Western Army in 1600 :
### Choices:
A. Akechi Mitsuhide
B. Mori Terumoto
C. Tokugawa Ieyasu
D. Ishida Mitsunari""")
response

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'The answer is **B. Mori Terumoto.**\n\nThe provided text states that "They then also appointed Mōri Terumoto to be the overall commander. They formed what came to be referred to as the Western Army." This indicates that Mori Terumoto was the de-facto leader of the Western Army in 1600.'